In [1]:
HTML(read(open("style.html"), String))

HTML{String}("<link href='https://fonts.googleapis.com/css?family=Alegreya+Sans:100,300,400,500,700,800,900,100italic,300italic,400italic,500italic,700italic,800italic,900italic' rel='stylesheet' type='text/css'>\r\n<link href='https://fonts.googleapis.com/css?family=Arvo:400,700,400italic' rel='stylesheet' type='text/css'>\r\n<link href='https://fonts.googleapis.com/css?family=PT+Mono' rel='stylesheet' type='text/css'>\r\n<link href='https://fonts.googleapis.com/css?family=Shadows+Into+Light' rel='stylesheet' type='text/css'>\r\n<link href='https://fonts.googleapis.com/css?family=Philosopher:400,700,400italic,700italic' rel='stylesheet' type='text/css'>\r\n\r\n<style>\r\n\r\n@font-face {\r\n    font-family: \"Computer Modern\";\r\n    src: url('https://mirrors.ctan.org/fonts/cm-unicode/fonts/otf/cmunss.otf');\r\n}\r\n\r\n.container { width: 100% }\r\n\r\n/* Formatting for header cells */\r\n.text_cell_render h1 {\r\n    font-family: 'Philosopher', sans-serif;\r\n    font-weight: 400;\r\n    font-size: 2.2em;\r\n    line-height: 100%;\r\n    color: rgb(0, 80, 120);\r\n    margin-bottom: 0.1em;\r\n    margin-top: 0.1em;\r\n    display: block;\r\n}\t\r\n.text_cell_render h2 {\r\n    font-family: 'Philosopher', serif;\r\n    font-weight: 400;\r\n    font-size: 1.9em;\r\n    line-height: 100%;\r\n    color: rgb(200,100,0);\r\n    margin-bottom: 0.1em;\r\n    margin-top: 0.1em;\r\n    display: block;\r\n}\t\r\n\r\n.text_cell_render h3 {\r\n    font-family: 'Philosopher', serif;\r\n    margin-top:12px;\r\n    margin-bottom: 3px;\r\n    font-style: italic;\r\n    color: rgb(94,127,192);\r\n}\r\n\r\n.text_cell_render h4 {\r\n    font-family: 'Philosopher', serif;\r\n}\r\n\r\n.text_cell_render h5 {\r\n    font-family: 'Alegreya Sans', sans-serif;\r\n    font-weight: 300;\r\n    font-size: 16pt;\r\n    color: grey;\r\n    font-style: italic;\r\n    margin-bottom: .1em;\r\n    margin-top: 0.1em;\r\n    display: block;\r\n}\r\n\r\n.text_cell_render h6 {\r\n    font-family: 'PT Mono', sans-serif;\r\n    font-weight: 300;\r\n    font-size: 10pt;\r\n    color: grey;\r\n    margin-bottom: 1px;\r\n    margin-top: 1px;\r\n}\r\n\r\n.text_cell_render em {\r\n    font-family: 'Philosopher', sans-serif;\r\n    color:        blue;\r\n    background-color: rgb(255,220,180);\r\n    font-size:    110%;\r\n    margin-left:   2px;\r\n    margin-right:  2px;\r\n    font-weight:   100;\r\n}\r\n\r\n.text_cell_render b {\r\n    color:            rgb(255,195,195);\r\n    background-color: rgb(0,0,0);\r\n    font-size:    110%;\r\n    margin-left:   2px;\r\n    margin-right:  2px;\r\n    font-weight:   650;\r\n}\r\n\r\n.text_cell_render tt {\r\n    font-size:    120%;\r\n    margin-left:   2px;\r\n    margin-right:  2px;\r\n    font-weight:   150;\r\n}\r\n\r\n.Codemirror {\r\n    font-family: \"PT Mono\", sans-serif;\r\n    font-size: 100%;\r\n}\r\n\r\n</style>")

In [2]:
include("silent.jl")

@silent (macro with 1 method)

In [3]:
using Pkg
@silent Pkg.add("Chess")
import Chess

***

# Memoisierung

Memoisierung beschreibt die Methodik, bereits berechnete Informationen für eine spätere Verwendung zu speichern.

Im Zusammenhang mit Schach bietet es sich an, neue Züge oder Stellungen und deren Bewertung zu berechnen und anschließend zu speichern. Dies ermöglicht es verschiedenen Algorithmen, bsp. MiniMax oder der Alpha-Beta-Suche, neue Stellungen schneller zu berechnen. Stoßen die Algorithmen auf bekannte Stellungen, so muss nur der bereits berechnete Wert abgerufen werden. Dies spart sowohl Ressourcen als auch Zeit.
Wird eine inkrementelle Suche verwendet, so ermöglicht die ersparte Zeit eine tiefere Suche. Beispielsweise kann die Alpha-Beta-Suche bis zu einer Tiefe von 4 statt 3 in 5 Sekunden rechnen (Annahme).

Die berechneten Stellungen mit Bewertung müssen gespeichert werden. In diesem Notebook wird der [`Zobrist-Hash`](https://www.chessprogramming.org/Zobrist_Hashing) dazu verwendet.

Beim Zobrist-Hashing wird das Schachbrett auf einen `Integer`-Hash abgebildet. Dieser setzt sich aus folgenden Komponenten (Sub-Hashes) zusammen:
+ pieceHashes &rarr; ein Int für jede Figur auf jedem Feld (12 * 64)
+ colorBlackHash &rarr; ein Int der anzeigt, dass Schwarz am zug ist (1)
+ enPassantHashes &rarr; ein Int für jedes mögliche En-Passant-Feld (8)
+ castleHashes &rarr; ein Int für jedes Rochade-Recht (4)

Der Hash wird mit einem Wert von `0` initialisiert. Bei der weiteren Berechnung wird der erste Integer, aka die gespeicherte Information, mit dem Hash gexort. Der nächste Integer wird mit dem neuen Hash gexort. Das Prinzip wiederholt sich so lange bis alle Integer verxort wurden. Die Methodik des `XOR` wird verwendet, da es sehr einfach umkehrbar ist, indem nochmal mit dem gleichen Wert die `XOR`-Operation durchgeführt wird.

$ ((x \oplus  y) \oplus  y) = y $

Somit können iterative Änderungen leicht gespeichert werden, da nur der geänderte Wert auf den alten Hashwert gerechnet werden muss.

Die Anzahl aller möglichen Hashes bei einer Abbildung auf einen x-Bit Integer wird durch $2^{x}$  beschrieben. Bei einem 64-Bit Integer gibt es somit 18.446.744.073.709.551.616 Kombinationen. Während dies zunächst als genügend groß erscheint, ist bei einer Menge von $\sqrt(2^{x})$ bereits mit einer Hash-Kollision zu rechnen (Quelle: [chessprogramming.org/Zobrist_Hashing](https://www.chessprogramming.org/Zobrist_Hashing#Theory)).

Dies kann behoben werden, indem der Integer von 32-Bit auf 64-Bit, bzw. auf 128-Bit erweitert wird. Die folgende Implementation des Zobrist-Hash Verfahrens wird mit einem 64-Bit Hash durchgeführt.

Der Zobrist-Hash wird als `mutable struct` eingeführt, da die inkrementelle Berechnung den Hash verändern können muss. Die weiteren Bestandteile werden als `const` definiert, da sie nicht verändert werden sollen.

In [4]:
mutable struct ZobristHash
    const pieceHashes::Array{UInt64,2}
    const colorBlackHash::UInt64
    const enPassantHashes::Array{UInt64}
    const castleHashes::Array{UInt64}
    hash::UInt64
    
    function ZobristHash(board::Chess.Board)
        # assume all keys are unique for simplicity
        this = new(
            [rand(UInt64) for _ ∈ 1:14, __ ∈ 1:64], 
            rand(UInt64), 
            [rand(UInt64) for _ ∈ 1:64], 
            [rand(UInt64) for _ ∈ 1:4],
            0
        )
        rehash!(this, board)
        return this
    end
end

Der Transpositionstabllen-Datentyp wird als Alias `Transposition` definiert.

In [5]:
if !@isdefined(Transposition)
    const Transposition = Tuple{Int64, Int32}
end

Tuple{Int64, Int32}

Die Funktion `rehash!` berechnet und setzt den Hash neu, bzw. initial. Dabei werden alle Werte der Spielfiguren, Rochade-Rechte, En-Passant-Felder und Zugrechte iteriert.

**Input**:
+ zobrist &rarr; der Zobrist-Hash-Kontext
+ board &rarr; das Spielbrett

In [6]:
function rehash!(zobrist::ZobristHash, board::Chess.Board)
    zobrist.hash = 0
    for file ∈ 1:8
        for rank ∈ 1:8
            square = Chess.Square(Chess.SquareFile(file), Chess.SquareRank(rank))
            piece = Chess.pieceon(board, square)
            if piece !== Chess.EMPTY
                togglePiece!(zobrist, piece, square)
            end
        end
    end
    if Chess.sidetomove(board) === Chess.BLACK
        toggleColor!(zobrist)
    end
    toggleEnPassant!(zobrist, Chess.epsquare(board))
    if Chess.cancastlekingside(board, Chess.WHITE)
        toggleCastle!(zobrist, 1)
    end
    if Chess.cancastlequeenside(board, Chess.WHITE)
        toggleCastle!(zobrist, 2)
    end
    if Chess.cancastlekingside(board, Chess.BLACK)
        toggleCastle!(zobrist, 3)
    end
    if Chess.cancastlequeenside(board, Chess.BLACK)
        toggleCastle!(zobrist, 4)
    end
end

rehash! (generic function with 1 method)

Die Funktion `togglePiece!` verändert den Hashwert inkrementell, wenn eine Aktion einer Spielfigur durchgeführt wurde.

**Input**:
+ zobrist &rarr; der Zobrist-Hash-Kontext
+ piece &rarr; die Spielfigur die verändert wird
+ square &rarr; das Spielfeld der Spielfigur

**Output**
+ der neue Hash

In [7]:
function togglePiece!(zobrist::ZobristHash, piece::Chess.Piece, square::Chess.Square)
    zobrist.hash ⊻= zobrist.pieceHashes[piece.val, square.val]
end

togglePiece! (generic function with 1 method)

Die Funktion `toggleEnPassant!` verändert den Hashwert inkrementell, wenn eine Aktion ein En-Passant verursacht.

**Input**:
+ zobrist &rarr; der Zobrist-Hash-Kontext
+ square &rarr; das En-Passant-Feld

**Output**
+ der neue Hash

In [8]:
function toggleEnPassant!(zobrist::ZobristHash, square::Chess.Square)
    if square === Chess.SQ_NONE
        return
    end
    zobrist.hash ⊻= zobrist.enPassantHashes[square.val]
end

toggleEnPassant! (generic function with 1 method)

Die Funktion `toggleCastle!` verändert den Hashwert inkrementell, wenn eine Änderung der Rochade-Rechte durchgeführt wurde.

**Input**:
+ zobrist &rarr; der Zobrist-Hash-Kontext
+ index &rarr; der Index der Rochade (~könnte als Enum gelöst werden)
    + 1 &rarr; kurze Rochade (König-Seite, Weiß)
    + 2 &rarr; kurze Rochade (Königin-Seite, Weiß)
    + 3 &rarr; kurze Rochade (König-Seite, Schwarz)
    + 4 &rarr; kurze Rochade (Königin-Seite, Schwarz)

**Output**
+ der neue Hash

In [9]:
function toggleCastle!(zobrist::ZobristHash, index::Int)
    zobrist.hash ⊻= zobrist.castleHashes[index]
end

toggleCastle! (generic function with 1 method)

Die Funktion `togglePiece!` verändert den Hashwert inkrementell, wenn die aktive Spielfarbe sich ändert.

**Input**:
+ zobrist &rarr; der Zobrist-Hash-Kontext

**Output**
+ der neue Hash

In [10]:
function toggleColor!(zobrist::ZobristHash)
     zobrist.hash ⊻= zobrist.colorBlackHash
end

toggleColor! (generic function with 1 method)

***